In [1]:
import numpy as np
import time
import u12mod
import mmap
import numba

@numba.jit
def decode_uint12_ref(inp, out):
    """
    decode bytes from bytestring ``inp`` as 12 bit into ``out``
    """
    o = 0
    for i in range(0, len(inp), 3):
        s = inp[i:i + 3]
        a = s[0] | (s[1] & 0x0F) << 8
        b = (s[1] & 0xF0) >> 4 | s[2] << 4
        out[o] = a
        out[o + 1] = b
        o += 2
    return out

input_data = np.zeros(0x5758, dtype=bytes).tobytes()

out = np.zeros(930*16, dtype="uint16")
out2 = np.zeros(930*16, dtype="uint16")

decode_uint12_ref(input_data[40:], out)

print("out=", out)

u12mod.decode_uint12_cpp(inp=input_data[40:], out=out2)

print("out2=", out2)

for idx, (i, j) in enumerate(zip(out, out2)):
    if i != j:
        print(idx, i, j)
print(np.allclose(out, out2))

out= [0 0 0 ... 0 0 0]
out2= [0 0 0 ... 0 0 0]
True


In [2]:
%timeit decode_uint12_ref(input_data[40:], out)

29.4 µs ± 1.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
%timeit u12mod.decode_uint12_cpp(inp=input_data[40:], out=out2)

11.3 µs ± 324 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
%timeit u12mod.decode_uint12(inp=input_data[40:], out=out2)

117 ms ± 2.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
